### 필요한 라이브러리 로드

In [1]:
import xml.etree.ElementTree as ET
import os
import glob
import shutil
import random
import json
import cv2
import easyocr
from tqdm import tqdm

In [3]:
root_dir = "/home/pepsi/Downloads/mmdetection_data"
json_dir = glob.glob(root_dir + "/*.json")
xml_dir = glob.glob(root_dir + "/*.xml")
image_dir = glob.glob(root_dir + "/*.png")
output_dir = "/home/pepsi/Downloads/output"
split_ratio = 0.8

os.makedirs(output_dir, exist_ok=True)

In [4]:
train_img_dir = output_dir + "/images/train"
val_img_dir = output_dir + "/images/val"

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)

In [5]:
label_map = {
    "보행자": 0,
    "핑키": 1,
    "30제한": 2,
    "60제한": 3,
    "정지선": 4,
    "횡단보도": 5,
    "빨간불": 6,
    "노란불": 7,
    "초록불": 8
}

coco = {
    "images": [],
    "annotations": [],
    "categories": [{"id": v, "name": k} for k, v in label_map.items()]
}

image_id = 1
annotation_id = 1

reader = easyocr.Reader(['en'], gpu=True)

In [7]:
# 🔹 JSON 처리 (LabelMe + AIHub)
for json_path in tqdm(json_dir):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if "annotations" in data:  # AIHub
        file_name = data["filename"]
        width = int(data["camera"]["resolution_width"])
        height = int(data["camera"]["resolution_height"])
        annotations = data["annotations"]

    elif "shapes" in data:  # LabelMe
        file_name = data["imagePath"]
        width = int(data.get("imageWidth", 1920))
        height = int(data.get("imageHeight", 1080))
        annotations = data["shapes"]
    else:
        continue

    coco["images"].append({
        "id": image_id,
        "file_name": file_name,
        "width": width,
        "height": height
    })

    for ann in annotations:
        label = ann.get("label")
        if label not in label_map:
            continue

        pts = ann["points"]
        x = [p[0] for p in pts]
        y = [p[1] for p in pts]
        xmin, xmax = min(x), max(x)
        ymin, ymax = min(y), max(y)
        w, h = xmax - xmin, ymax - ymin

        coco["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": label_map[label],
            "bbox": [xmin, ymin, w, h],
            "area": w * h,
            "iscrowd": 0
        })
        annotation_id += 1
    image_id += 1

100%|██████████| 7104/7104 [00:02<00:00, 2384.32it/s]


In [11]:
# 🔹 XML 처리 (VOC)
for xml_path in tqdm(xml_dir):
    root = ET.parse(xml_path).getroot()
    file_name = root.findtext("filename")
    width = int(root.find("size/width").text)
    height = int(root.find("size/height").text)

    coco["images"].append({
        "id": image_id,
        "file_name": file_name,
        "width": width,
        "height": height
    })

    image_path = root_dir + "/" + file_name
    image = cv2.imread(str(image_path))

    for obj in root.findall("object"):
        name = obj.findtext("name")
        if name == "speedlimit":
            box = obj.find("bndbox")
            xmin = int(box.find("xmin").text)
            ymin = int(box.find("ymin").text)
            xmax = int(box.find("xmax").text)
            ymax = int(box.find("ymax").text)
            cropped = image[ymin:ymax, xmin:xmax]

            if cropped is None or cropped.size == 0 or cropped.shape[0] < 5 or cropped.shape[1] < 5:
                continue
            
            results = reader.readtext(cropped)
            detected_text = ''.join([res[1] for res in results])

            if "60" in detected_text:
                label = "60제한"
            elif "30" in detected_text:
                label = "30제한"
            else:
                continue
        else:
            continue  # 다른 라벨은 무시

        w, h = xmax - xmin, ymax - ymin
        coco["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": label_map[label],
            "bbox": [xmin, ymin, w, h],
            "area": w * h,
            "iscrowd": 0
        })
        annotation_id += 1

    image_id += 1

100%|██████████| 877/877 [00:37<00:00, 23.41it/s]


In [13]:
# 🔸 8:2 train/val 분리
random.shuffle(coco["images"])
split_idx = int(len(coco["images"]) * split_ratio)
train_imgs = coco["images"][:split_idx]
val_imgs = coco["images"][split_idx:]

train_ids = {img["id"] for img in train_imgs}
val_ids = {img["id"] for img in val_imgs}
train_anns = [a for a in coco["annotations"] if a["image_id"] in train_ids]
val_anns = [a for a in coco["annotations"] if a["image_id"] in val_ids]

# 🔸 JSON 저장
with open(output_dir + "/test/train.json", "w", encoding="utf-8") as f:
    json.dump({"images": train_imgs, "annotations": train_anns, "categories": coco["categories"]}, f, indent=4, ensure_ascii=False)

with open(output_dir + "/test/val.json", "w", encoding="utf-8") as f:
    json.dump({"images": val_imgs, "annotations": val_anns, "categories": coco["categories"]}, f, indent=4, ensure_ascii=False)

In [34]:
# 🔸 이미지 복사
def copy_images(image_list, dest_dir):
    for img in tqdm(image_list):
        src = "/home/pepsi/Downloads/mmdetection_data/" + img["file_name"]
        dst = dest_dir + "/" + img["file_name"]
        shutil.copyfile(src, dst)

copy_images(train_imgs, train_img_dir)
copy_images(val_imgs, val_img_dir)

100%|██████████| 1597/1597 [00:23<00:00, 69.18it/s] 
